# Automatic credibility assessment of popular science articles available online

## Module 1: Topic modelling

In [129]:
# Load dependencies

import numpy as np
import nltk
import re
import os
import codecs

from morfeusz import analyse
import gensim
import gensim.corpora as corpora


import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

# helper class for visualization purposes
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Load polish tokenizer from NLTK

In [111]:
polish_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

### Load raw article texts

In [118]:
def raw_text_list(path_to_articles='articles'):
    articles = []
    labels = []

    for filename in os.listdir(path_to_articles):
        article = ""

        if not filename.startswith('.') and filename.endswith(".txt"): 
            # print(os.path.join(directory, filename))
            f = open(path_to_articles + "/" + filename)
            lines = f.readlines()
            article = " ".join(lines)
            labels.append(filename)
            articles.append(article)

    return articles, labels

# print first elements of 2 parallell lists from 2xN array (where N is the number of articles)

articles, labels = raw_text_list()

for i in range(0,7):
    print(articles[i][0:100] + '...')
    print(labels[i] + '\n')

 ﻿Loratadyna jest lekiem wprowadzonym na rynek w 1993 r. pod nazwą Claritine przez koncern Schering-...
29.txt

 ﻿Mięśniaki – najczęstsze łagodne guzy macicy Mięśniaki są to guzy łagodne, wywodzące się z włókien ...
15.txt

 Co roku zaleca się ludziom przyjmowanie szczepionki przeciw grypie, co ma oszczędzić im powtarzając...
114.txt

 Jednym z podstawowych elementów edukacji moich dietetycznych pacjentów jest nauczenie interpretowan...
100.txt

 Medycyna ludowa od dawna wykorzystuje lecznicze właściwości pestek dyni.
  Używa się ich, zarówno u...
128.txt

 Żywienie Optymalne zapewnia wszystkim, którzy je stosują, eliminowanie przyczyn chorób.
  Jest to j...
129.txt

 Ludzki organizm, głównie w wątrobie, sam wytwarza cholesterol, bo jest on nam potrzebny.
  W zdrowo...
101.txt



### Tokenization & stemming

In [123]:
def base_word(word):
    return analyse(word)[0][0][1]

def extract_words_from_string(s):
    return re.findall(re.compile('\w+'), s.lower())

def article_vocabulary_stemmed(article):
    '''
        'article' is of type string
    '''

    word_list = extract_words_from_string(article)
    article_vocabulary = []

    for w in word_list:
        try:
            a = base_word(w)
            article_vocabulary.append(a)
        except KeyError as e:
            pass

    return article_vocabulary

def data_lemmatized(articles):
    articles_lemmatized = []
    for article in articles:
        articles_lemmatized.append(article_vocabulary_stemmed(article))
        
    return articles_lemmatized


### Get the article vocabulary

In [120]:
articles_lemmatized = data_lemmatized(articles)
articles_lemmatized[:1]

[['loratadyna',
  'być',
  'lek',
  'wprowadzić',
  'na',
  'rynek',
  'w',
  'r',
  'pod',
  'nazwać',
  'claritine',
  'przez',
  'koncern',
  'schering',
  'plough',
  'europ',
  'ten',
  'popularny',
  'farmaceutyk',
  'o',
  'działanie',
  'przeciwalergiczny',
  'oraz',
  'przeciwhistaminowy',
  'w',
  'jak',
  'sposób',
  'dziać',
  'loratadyna',
  'loratadyna',
  'wykazywać',
  'selektywny',
  'antagonistyczny',
  'działanie',
  'na',
  'obwodowy',
  'receptor',
  'h1',
  'lek',
  'należeć',
  'stosować',
  'doustnie',
  'ponieważ',
  'dobrze',
  'oraz',
  'szybko',
  'wchłaniać',
  'się',
  'z',
  'przewód',
  'pokarmowy',
  'podczas',
  'pierwsze',
  'przejście',
  'przez',
  'wątroba',
  'być',
  'niemal',
  'całkowicie',
  'metabolizować',
  'przy',
  'udział',
  'cyp2d6',
  'oraz',
  'cyp3a4',
  'desloratadyna',
  'być',
  'główny',
  'metabolit',
  'czynny',
  'farmakologicznie',
  'odpowiedzialny',
  'za',
  'odpowiedni',
  'działanie',
  'lek',
  'wysoki',
  'stężenie',


### Term document frequency

In [121]:
id2word = corpora.Dictionary(articles_lemmatized)

articles_lem = articles_lemmatized

corpus = [id2word.doc2bow(article) for article in articles_lem]

import gensim.downloader as api
from gensim.sklearn_api import TfIdfTransformer
import pickle

model = TfIdfTransformer(id2word=id2word) # -- use for the first time
tfidf_corpus = model.fit_transform(corpus)
# model = picle.load(open('tfidf_medical_corpus.p', 'rb')) 
# pickle.dump(model, open('tfidf_medical_corpus.p', 'wb')) # -- use if model has already been pickled


# Use the model to show retrieved context

In [130]:
import numpy as np

n = 243 # first document's ID
imp = 5 # amount of the most important words for the document

def printParagraphWithContext(n=n,imp=imp):
    for i in range (n-5, n+5):
        indexes, scores = [list(a) for a in zip(*tfidf_corpus[i])]

        np_scores = np.array(scores) # transform scores to numpy array to use the sorting function
        idxs_most_important_words = np_scores.argsort()[-imp:] 

        context = ''
        for idx in reversed(idxs_most_important_words):
            context += (id2word[indexes[idx]] + ' ')

        print('\n')
        print(color.BOLD + context + color.END)
        print(articles[i][0:240] + '...')
        
printParagraphWithContext(n,imp)



grzybica pochwa drożdżak intymny candida 
 Grzybicę pochwy zwana także drożdżycą choć raz w życiu miała co czwarta kobieta.
  Rzadziej grzyby atakują miejsca intymne mężczyzn.
  Choroba może się rozwijać, mimo że dbamy o higienę intymną.
  Niestety, grzybica pochwy ma też tendencję...


tłuszcz węglowodan energia glukoza żywienie 
 Genotyp człowieka został ukształtowany w czasach paleolitu, a nasi przodkowie żywili się jedynie tym, co upolowali, zebrali albo wygrzebali z ziemi, stąd podstawowym składnikiem ich pożywienia było mięso różnego rodzaju wraz z tłuszczem.
 ...


rak chirurgiczny interwencja przeżywalność chemioterapia 
 Medycyna jest bardzo dumna z rygorystycznych metod badawczych stanowiących podstawę leczenia raka.
  Jeśli u kogoś zdiagnozowano raka, osoba ta z miejsca staje przed ogromnym naciskiem ze strony systemu ochrony zdrowia, który wymusza na ni...


wodorowęglan pH sód łyżeczka cez 
 Termin ’’wysokie pH’’ oznacza "silnie alkaliczny", co w leczeniu raka odnosi się

## Module 2: WordNet & Hyperonymy (meaning aggregates)

### plWordNet XML parser

In [3]:
import xml.etree.ElementTree as ET
import pickle

### Load WNet XML to the parser
tree = pickle.load(open('plwntree.dump', 'rb'))
PATH_TO_WN = 'plwntree.dump'

# For the first run -->

# tree = ET.parse('plwordnet-4.0.xml')
# pickle.dump(tree, open('plwntree.dump', 'w'))


### Load lexical unit <-> sysnet mapping in Python readable format
synunitmap = pickle.load(open('synunitmap.dump', 'rb'))

# for the first run -->

# synunitmap = synsetUnitMapping() # for the first run
# pickle.dump(synunitmap, open('synunitmap.dump', 'wb'))


root = tree.getroot()

# def synsetUnitMapping(root=root):
#     synsets = dict()
#     for child in root.iter():
#         if child.tag == "synset":
#             synset_id = child.attrib['id']
#             unit_list = list(child.findall("unit-id"))
#             synsets[synset_id] = [uid.text for uid in unit_list]
            
#     return synsets

# def printNFirstMappings(n=100, synunitmap=synunitmap):
#     from heapq import nlargest

#     for key in nlargest(10, synunitmap, key=synunitmap.get):
#         print(key, synunitmap[key])
    
# def wordFromId(word_id):
#     for child in root.findall('lexical-unit'):
#         if child.attrib['id'] == word_id:
#             return child.attrib['name']
#     raise ValueError("No such lexical unit ID in plWordNet")
                        
    
# def idFromWord(word):
#     for child in root.findall('lexical-unit'):
#         if child.attrib['name'] == word:
#             return child.attrib['id']
#     raise ValueError("No such lexical unit in plWordNet")
    

# def synsetsForWord(word='', synunitmap = synunitmap, word_id='', tagtype='name'):
#     ''' 
#     Find synset for a word (given by id or name).
#     The loop iterates through synset - lex. unit mapping.
#     '''
#     if tagtype == 'name':
#         word_id = idFromWord(word)
    
#     synsets = []
#     for s in synunitmap.keys():
#         if word_id in synunitmap[s]:
#             synsets.append(s)
    
#     return synsets


# def getHyponymsFromWord(word, synunitmap = synunitmap):
#     synsets_for_word = synsetsForWord(word=word)
#     hypo_synsets = []
    
#     for child in root.findall('synsetrelations'):
#         if child.attrib['relation'] in ['11', '211'] and child.attrib['parent'] in synsets_for_word:
#             hypo_synsets.append(child.attrib['child'])
    
#     hyponyms = []
#     for s in hypo_synsets:
#         hyponyms = list(set().union(hyponyms,synunitmap[s]))
    
#     return [wordFromId(word_id) for word_id in hyponyms]


In [106]:
getHyponymsFromWord('objaw chorobowy')

['ostry brzuch',
 'hiperprolaktynemia',
 'anhedonia',
 'zespół suchego oka',
 'abulia',
 'leukocytoza',
 'kulawizna',
 'upośledzenie',
 'ambitendencja',
 'objaw Aschaffenburga',
 'biegunka',
 'zmiana chorobowa',
 'alalia',
 'swędzenie',
 'wytrzeszcz złośliwy',
 'eozynofilia',
 'nosicielstwo',
 'autoskopia',
 'perseweracja',
 'wstrząs',
 'puchlina wodna',
 'rozwolnienie',
 'bielmo',
 'azoospermia',
 'entropium',
 'mioza',
 'niedokrwienie',
 'kwasica',
 'leukokoria',
 'wzdęcie',
 'nadciśnienie',
 'męczliwość',
 'bakteriomocz',
 'słowotok',
 'aura',
 'bezmocz',
 'ektropion',
 'odrętwienie',
 'paroksyzm',
 'światłowstręt',
 'inkoherencja',
 'wodogłowie',
 'afonia',
 'koprolalia',
 'ruptura',
 'gorączka',
 'pasma Meesa',
 'małopłytkowość',
 'schizofazja',
 'skąpomocz',
 'stawy Charcota',
 'leukopenia',
 'zawał',
 'ciągotka',
 'bakteriuria',
 'chloroza',
 'stan spastyczny',
 'objaw zwiastunowy',
 'chrypa',
 'majaczenie',
 'palpitacja',
 'bezwład',
 'hipertrofia',
 'oftalmoplegia',
 'skleroza

## Load sentences and visualize then in a window

In [17]:
import pickle

### Load

In [138]:
'''
    Load sentences as a list of dicts:
    [
        ...,
        { 'body': 'sentence body',
          'contextual_rate': 1,
          'uncontextual_rate': -1,
          'article_id': 10,
          'position_in_article': 3},
        ...
    ]
'''
SENTENCES = pickle.load(open('sentences_dict.dump', 'rb'))

### Visualize

In [166]:
LEFT_WINDOW_SIZE = 2
RIGHT_WINDOW_SIZE = 1

def is_inassessible(sent):
    return sent['uncontextual_rate'] == '-1'
        
def get_article(art_id, padding, sentences=SENTENCES):
    art = []
    for sent in sentences:
        if sent['article_id'] == art_id:
            art.append(sent['body'])
            
    return ['']*padding + art + ['']*padding
                
def getSentWithWindow(sent, lw=3, rw=1, sentences=SENTENCES):
    pos = sent['position_in_article']
    padding = max(lw, rw)
    begin = pos - lw + padding
    end = pos + rw + padding
    art = get_article(sent['article_id'], padding)
    res = [color.BOLD + s + color.END if art.index(s) + 1 - padding == pos else s for s in art[begin:end]]
    
    return res

def getContext(sent, sentences=SENTENCES):
    sent_with_window = getSentWithWindow(sent)
    only_context = []
    for s in sent_with_window:
        if '\033[1m' not in s:
            only_context.append(s)
    return only_context
    

### Get inassessable sentences

In [168]:
def getInassessibleSentences(sentences=sentences):
    inass = []
    for sent in sentences:
        if is_inassessible(sent):
            inass.append(sent)
    return inass

inassessible = getInassessibleSentences()

# Save context paragraphs in a corresponding directory

# for i, sent in enumerate(inassessible):
#     paragraph = " ".join(getSentWithWindow(sent)).replace('\033[1m', '').replace('\033[0m', '')
#     filename = str(i) + ".txt"
#     f = open( os.path.join('paragraphs', filename), 'w')
#     f.write(paragraph)

# Print paragraphs

for i, sent in enumerate(inassessible):
    context_window = getSentWithWindow(sent)
    for s in context_window:
        print(s)
    print('\n')

 Według autora pracy zamieszczonej na łamach pisma "Ultrasound", traktowanie tego badania jako głównego wskaźnika wad genetycznych skutkuje utratą wielu prawidłowych ciąż.
 Doktor Hylton Meire, autor publikacji, rozważa, czy badanie ultrasonograficzne jest aż tak wiarygodne, jak się powszechnie przyjmuje.
 Głównym przedmiotem wątpliwości naukowca jest brak danych naukowych na temat wykonywania pomiaru przezierności karku płodu jako rutynowego testu przesiewowego na obecność wad genetycznych, zwłaszcza zespołu Downa.
 Badanie przezierności karku jest wykonywane podczas USG położniczego pomiędzy 10 a 14 tygodniem ciąży.


 Głównym przedmiotem wątpliwości naukowca jest brak danych naukowych na temat wykonywania pomiaru przezierności karku płodu jako rutynowego testu przesiewowego na obecność wad genetycznych, zwłaszcza zespołu Downa.
 Badanie przezierności karku jest wykonywane podczas USG położniczego pomiędzy 10 a 14 tygodniem ciąży.
 Mierzy się ilość płynu zbierającego się z tyłu szyi 

 Wszyscy skupiają się wyłącznie na szczepionkach, ponieważ autyzm często pokazuje się dopiero po rozpoczęciu rutynowego programu szczepień.
 Zdrowe dziecko do tego czasu wygląda na zdrowe.
 Rozmyślnie używam słowo „wygląda”.
 Jak wskazałem w ostatnim biuletynie, że Kuba, która ma wskaźnik szczepień około 95% (jeden z najwyższych na świecie), wciąż ma bardzo mało przypadków autyzmu, w porównaniu z Europą czy USA.


 Na podstawie tych badań prof. Ruo Feng z Nanjing University, w 2000 opublikował takie wytyczne: „Komercyjne lub edukacyjne obrazowanie płodu przez USG powinno być ściśle eliminowane.
 USG do identyfikacji płci płodu i obrazowania położenia płodu powinno się ściśle eliminować.
 Dla najlepszej wczesnej ciąży unikać USG„.
 Dr Feng mówi bardzo wyraźnie: unikać – to nie wyraz dwuznaczny! Książka Jima Westa The Chinese Human Studies nie jest dostępna na rynku.


 Dr Feng mówi bardzo wyraźnie: unikać – to nie wyraz dwuznaczny! Książka Jima Westa The Chinese Human Studies nie jest d

 Czułam się, jakbym wracała z wojny, choć na wojnie nigdy nie byłam.
 Jakby zeszło ze mnie całe powietrze! Reakcje naszego organizmu na takie emocje są nieprzewidywalne, więc warto, by był ktoś kto poda rękę.
 Kto postawi nas do pionu! Na badanie nie trzeba się w żaden inny niż psychiczny sposób przygotowywać.
 A potem przetrwać brutalny okres oczekiwania, który minimalnie wynosi 2 tygodnie.


 Jakby zeszło ze mnie całe powietrze! Reakcje naszego organizmu na takie emocje są nieprzewidywalne, więc warto, by był ktoś kto poda rękę.
 Kto postawi nas do pionu! Na badanie nie trzeba się w żaden inny niż psychiczny sposób przygotowywać.
 A potem przetrwać brutalny okres oczekiwania, który minimalnie wynosi 2 tygodnie.
 Jeśli chcemy otrzymać wyniki szybciej można pokusić się o placówkę, która bada materiał metodą FISH-a, ale trzeba pamiętać, że nie każdą próbkę można tak badać.


 To oceni osoba przyjmująca od nas materiał.
 No i jest to już opcja płatna.
 Wyniki jednak można otrzymać już po


 Sprawdź w naszej galerii, jakie są objawy cukrzycy.
 Jak się przed nią bronić? Zmianą stylu życia.
 W tym może ci pomóc aplikacja Asystent Zdrowego Żywienia, którą przygotowali eksperci IŻŻ.
 Permanentne zmęczenie Cukrzyca to choroba metaboliczna, w przebiegu której we krwi utrzymuje się podwyższony poziom cukru.


 Permanentne zmęczenie Cukrzyca to choroba metaboliczna, w przebiegu której we krwi utrzymuje się podwyższony poziom cukru.
 Do niedawna lekarze wyróżniali dwa typy cukrzycy: 1 i 2.
 Pierwszy, gdy organizm niszczy komórki trzustki tak, że nie produkuje ona insuliny.
 Drugi charakteryzujący się nie niedoborem insuliny, tylko brakiem wrażliwości komórek organizmu na jej działanie.


 Do niedawna lekarze wyróżniali dwa typy cukrzycy: 1 i 2.
 Pierwszy, gdy organizm niszczy komórki trzustki tak, że nie produkuje ona insuliny.
 Drugi charakteryzujący się nie niedoborem insuliny, tylko brakiem wrażliwości komórek organizmu na jej działanie.
 Według najnowszych kryteriów Polskiego

 Pierwsze objawy HIV są zwykle niespecyficzne i mogą przypominać grypę lub zwykłe przeziębienie.


 Pierwsze objawy HIV są zwykle niespecyficzne i mogą przypominać grypę lub zwykłe przeziębienie.
 Jak rozpoznać pierwsze objawy HIV i AIDS? HIV to wirus upośledzenia odporności, który uszkadza układ immunologiczny człowieka.
 Po wniknięciu do organizmu atakuje pewne typy białych ciałek krwi.
 W ten sposób powoli wyniszcza układ odpornościowy chorego.


 Po wniknięciu do organizmu atakuje pewne typy białych ciałek krwi.
 W ten sposób powoli wyniszcza układ odpornościowy chorego.
 W efekcie organizm staje się bezbronny nawet wobec pozornie niegroźnych infekcji, z którymi kiedyś radził sobie bez problemu.
 Stan bezbronności immunologicznej organizmu to AIDS - zespół nabytego niedoboru odporności.


 W efekcie organizm staje się bezbronny nawet wobec pozornie niegroźnych infekcji, z którymi kiedyś radził sobie bez problemu.
 Stan bezbronności immunologicznej organizmu to AIDS - zespół nabyteg

...

 Są to owoce, które warto uzwględnić w codziennej diecie dr Budwig.



  Ciężkie niedotlenienie okołoporodowe dotyczy szacunkowo jednego na tysiąc noworodków.
 Ale o niewielkim niedotlenieniu okołoporodowym noworodka słyszymy zdecydowanie częściej - na porodówce mówi się o nim niemal tak często jak o żółtaczce fizjologicznej, którą stwierdza się u prawie połowy zdrowych dzieci.
 Przeczytaj, skąd się bierze niedotlenienie okołoporodowe, czym skutkuje i jak można mu zapobiec oraz w jaki sposób leczyć.
 Niedotlenienie okołoporodowe noworodka to prawdziwa zmora przyszłych mam, które obawiają się, że mimo prawidłowo przebiegającej ciąży, maksymalnej troski o zdrowie swoje i dziecka, coś na finiszu może się skomplikować i w zaledwie kilka minut doprowadzić do tego, że noworodek nie urodzi w pełni zdrowy.


 Niedotlenienie okołoporodowe noworodka to prawdziwa zmora przyszłych mam, które obawiają się, że mimo prawidłowo przebiegającej ciąży, maksymalnej troski o zdrowie swoje i dziecka, coś na

 Nie przeprowadzono odpowiednich badań, ale faktem jest, że w wyniku przeszczepu kału od osoby otyłej "obdarowany" utył i odwrotnie.
 Dlatego dziś tak starannie dobiera się dawców.


 A co z przeszczepem z innych wskazań, np. w leczeniu otyłości? Faktycznie, pojawiają się takie pomysły, ale to wciąż medycyna alternatywna.
 Nie przeprowadzono odpowiednich badań, ale faktem jest, że w wyniku przeszczepu kału od osoby otyłej "obdarowany" utył i odwrotnie.
 Dlatego dziś tak starannie dobiera się dawców.
 Jak? Dawca musi być zdrowy, przejść szereg badań na obecność wirusów wątroby, pasożytów, HIV czy bakterii patogennych.


 Dlatego dziś tak starannie dobiera się dawców.
 Jak? Dawca musi być zdrowy, przejść szereg badań na obecność wirusów wątroby, pasożytów, HIV czy bakterii patogennych.
 Powinien prawidłowo ważyć, a im więcej pacjent o nim wie, tym lepiej.
 Często dawcami są krewni.


 To pozwala uniknąć odrzutu? Przeszczep kału nie wiąże się z ryzykiem odrzutu, bo tak naprawdę nie jest t

 W jednym badaniu wykazano że dawka 1100 IU dziennie, wraz z wapniem, zmniejszyła ryzyka wystąpienia raka o 60% ( 32 , 33 ).
 Depresja: Badania wykazały że witamina D powoduje łagodną redukcję nasilenia objawów u osób z depresją ( 34 ).


 Cukrzyca typu 1: W jednym z badań stwierdzono że dawka 2000 IU witaminy D dziennie u niemowląt zmniejsza ryzyko wystąpienia cukrzycy typu 1 o 78% ( 35 ).
 Śmiertelność: Niektóre badania wykazały że suplementacja witaminy D zmniejsza ryzyko śmierci podczas pobytu w szpitalach, badaniach i diagnozowaniu lekarskim co oznacza że może pomóc żyć dłużej.
 To jest tylko wierzchołek góry lodowej.
 Niski poziom witaminy D są związane z różnego rodzaju chorobami i suplementacja wykazała liczne korzyści dla zdrowia.


 Jaka dawka witaminy D3 będzie właściwa? Jedynym pewnym sposobem żeby to ustalić jest zbadanie poziomu pochodnej witaminy D: 25(OH)D w krwi.
 Badanie zmierzy formę przechowywania witaminy D - 25(OH)D.
 Obecnie uważa się że stężenie 12 ng/ml (30 nmo



 Biorezonans jest metodą wymyśloną na początku XX w, związaną z filozofią medycyny chińskiej, a w szczególności akupresurą.
 Według osób stosujących ten rodzaj terapii może ona pomóc na pasożyty, bakterie, boreliozę, a nawet uzależnienia.
 Biorezonansu, który nie jest metodą naukową, nie należy mylić z rezonansem magnetycznym – sprawdzonym badaniem szeroko wykorzystywanym w medycynie do diagnostyki i umożliwiającym uzyskiwanie wewnętrznych obrazów ciała.


 Konieczne jest trzymanie dwóch metalowych końcówek, które podłączone są do aparatury, podczas gdy przez ciało człowieka przepuszczany jest prąd o różnych częstotliwościach (ale niskim natężeniu).
 Można w tym czasie odczuwać nieprzyjemne mrowienie w rękach lub rozdrażnienie.
 Całość może być dość czasochłonna – może trwać nawet do dwóch godzin.
 Długość badania zależy od rozległości problemów zdrowotnych.


 Całość może być dość czasochłonna – może trwać nawet do dwóch godzin.
 Długość badania zależy od rozległości problemów zdrow

 Na co dzień możesz używać żeli do higieny intymnej, które posiadają pH zbliżone do odczynu miejsc intymnych (5,2-5,5).
 Używaj osobnego ręcznika do higieny intymnej Do mycia okolic intymnych nie używaj myjek ani gąbek, ponieważ są siedliskiem grzybów i bakterii.
 Najlepiej, aby do wycierania okolic intymnych używać jednorazowych, papierowych ręczników.
 Jeśli jednak nie masz takiej możliwości, użyj bawełnianego, ale przeznaczonego tylko do tego celu.


 Używaj osobnego ręcznika do higieny intymnej Do mycia okolic intymnych nie używaj myjek ani gąbek, ponieważ są siedliskiem grzybów i bakterii.
 Najlepiej, aby do wycierania okolic intymnych używać jednorazowych, papierowych ręczników.
 Jeśli jednak nie masz takiej możliwości, użyj bawełnianego, ale przeznaczonego tylko do tego celu.
 Dbaj, by zawsze był suchy (grzyby bardzo szybko rozmnażają się w wilgoci).


 Najlepiej, aby do wycierania okolic intymnych używać jednorazowych, papierowych ręczników.
 Jeśli jednak nie masz takiej możliw

# Terms from the corpus

In [84]:
import csv

def createListOfTerms(filename='terms.csv'):
    with open(filename) as terms:
        list_of_terms = list()
        csv_terms = csv.reader(terms, delimiter='\t')
        for row in csv_terms:
            list_of_terms.append(row)
        return list_of_terms
            
list_of_terms = createListOfTerms()

In [134]:
hyperonym_candidates = [term[2] for term in list_of_terms] # lemmatized terms list

# sentences_stemmed_tokenized = []
# for sent in inassessible:
#     sentences_stemmed_tokenized.append(' '.join(tokenize_and_stem(sent['body'])))

def getSentencesContainingHyperonymCandidate(sentences, hyperonym_candidates):
    sentences_with_hyperonym = set()
    for sent in sentences:
        sent_stemmed = ' '.join(tokenize_and_stem(sent['body']))
        for h in hyperonym_candidates:
            if h in sent_stemmed:
                sentences_with_hyperonym.add((sent['body'], sent['article_id'], sent['position_in_article']))
         
    return sentences_with_hyperonym

def getExpressionWithRelatedSentences(sentences, list_of_expressions):
    '''
    {
        'article_id': id artykułu, z którego pochodzi zdanie, które zawiera dany hiperonim
        'expression': wyrażenie
        'index': indeks stringa ZLEMATYZOWANEGO zdania, na którym występuje wyrażenie hiperonimiczne
        'pos_in_article': pozycja zdania w artykule (które to zdanie z kolei)
    }
    '''
    hyperonyms_with_related_sentences =[]
    for sent in sentences:
        sent_stemmed = ' '.join(tokenize_and_stem(sent['body']))
        for h in hyperonym_candidates:
            try:
                hyperonyms_with_related_sentences.append({
                    'article_id': sent['article_id'], 
                    'pos_in_article': sent['position_in_article'],
                    'expression': h,
                    'index': sent_stemmed.index(h)})
            except ValueError:
                pass
    
    return hyperonyms_with_related_sentences


sentences_with_aggregates = getSentencesContainingHyperonymCandidate(inassessible, hyperonym_candidates)
hyp_with_sents = getHyperonymsWithRelatedSentences(inassessible, hyperonym_candidates)

# TODO
# 
# Z sentences_dict pobieramy
# otoczenie zdania, znowu lematyzujemy/tokenizujemy i wybieramy kandydatów na hiponimy. 
#
# Pobieramy hiponimy kandydatów na hiperonimy ze słowosieci (depth=2).
# Znajdujemy przecięcie zbiorów hipo-kandydatów i kandydatów na hipo.

In [135]:
print(len(sentences_with_aggregates))
print(len(inassessible))
# sent_with_aggregates_list = list(sentences_with_aggregates)
# sent_with_aggregates_list[100:200]
hyp_with_sents

2662
3035


[{'article_id': '40\n',
  'hyperonym': 'zespół Down',
  'index': 177,
  'pos_in_article': 5},
 {'article_id': '40\n', 'hyperonym': 'rak', 'index': 43, 'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'test',
  'index': 123,
  'pos_in_article': 5},
 {'article_id': '40\n', 'hyperonym': 'D', 'index': 184, 'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'zespół',
  'index': 177,
  'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'płód',
  'index': 104,
  'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'naukowiec',
  'index': 28,
  'pos_in_article': 5},
 {'article_id': '40\n', 'hyperonym': 'brak', 'index': 42, 'pos_in_article': 5},
 {'article_id': '40\n', 'hyperonym': 'dana', 'index': 47, 'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'obecność',
  'index': 143,
  'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'przezierność kark',
  'index': 86,
  'pos_in_article': 5},
 {'article_id': '40\n',
  'hyperonym': 'dziec